In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.2/757.2 kB 8.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
import torch
from ultralytics import YOLO
import torch.nn.utils.prune as prune

# Load the original YOLO model
model = YOLO('/content/drive/MyDrive/HackIA/ModuleEdge/models/best.pt')

# Apply pruning (example: using L1 unstructured pruning)
for name, module in model.model.named_modules():
    if isinstance(module, torch.nn.Conv2d):
        prune.l1_unstructured(module, name='weight', amount=0.2)

# Remove the pruning reparametrization to store the model in a normal state
for name, module in model.model.named_modules():
    if isinstance(module, torch.nn.Conv2d):
        prune.remove(module, 'weight')

# Save the entire pruned model
torch.save(model.model, '/content/drive/MyDrive/HackIA/ModuleEdge/models/pruned_model_yolo.pt')


In [24]:
import torch
from ultralytics import YOLO
import torch.nn.utils.prune as prune

# Load the original YOLO model
model = YOLO('/content/drive/MyDrive/HackIA/ModuleEdge/models/best.pt')  # Ensure you have the correct path and model type

# Apply pruning (example: using L1 unstructured pruning)
for name, module in model.model.named_modules():
    if isinstance(module, torch.nn.Conv2d):
        prune.l1_unstructured(module, name='weight', amount=0.2)

# Remove the pruning reparametrization to store the model in a normal state
for name, module in model.model.named_modules():
    if isinstance(module, torch.nn.Conv2d):
        prune.remove(module, 'weight')

# Save the pruned model state dictionary
torch.save(model.model.state_dict(), '/content/drive/MyDrive/HackIA/ModuleEdge/models/pruned_model_yolo_state_dict.pt')


In [23]:
from ultralytics import YOLO
import torch
from ultralytics.nn.modules import Bottleneck, Conv, C2f, SPPF, Detect
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

# Load a model
yolo = YOLO("/content/drive/MyDrive/HackIA/ModuleEdge/models/best.pt")  # build a new model from scratch
model = yolo.model

ws = []
bs = []

for name, m in model.named_modules():
    if isinstance(m, torch.nn.BatchNorm2d):
        w = m.weight.abs().detach()
        b = m.bias.abs().detach()
        ws.append(w)
        bs.append(b)
        print(name, w.max().item(), w.min().item(), b.max().item(), b.min().item())
# keep
factor = 0.5
ws = torch.cat(ws)
threshold = torch.sort(ws, descending=True)[0][int(len(ws) * factor)]
print(threshold)


def prune_conv(conv1: Conv, conv2: Conv):
    gamma = conv1.bn.weight.data.detach()
    beta  = conv1.bn.bias.data.detach()

    keep_idxs = []
    local_threshold = threshold
    while len(keep_idxs) < 8:
        keep_idxs = torch.where(gamma.abs() >= local_threshold)[0]
        local_threshold = local_threshold * 0.5
    n = len(keep_idxs)
    # n = max(int(len(idxs) * 0.8), p)
    print(n / len(gamma) * 100)
    # scale = len(idxs) / n
    conv1.bn.weight.data = gamma[keep_idxs]
    conv1.bn.bias.data   = beta[keep_idxs]
    conv1.bn.running_var.data = conv1.bn.running_var.data[keep_idxs]
    conv1.bn.running_mean.data = conv1.bn.running_mean.data[keep_idxs]
    conv1.bn.num_features = n
    conv1.conv.weight.data = conv1.conv.weight.data[keep_idxs]
    conv1.conv.out_channels = n

    if conv1.conv.bias is not None:
        conv1.conv.bias.data = conv1.conv.bias.data[keep_idxs]

    if not isinstance(conv2, list):
        conv2 = [conv2]

    for item in conv2:
        if item is not None:
            if isinstance(item, Conv):
                conv = item.conv
            else:
                conv = item
            conv.in_channels = n
            conv.weight.data = conv.weight.data[:, keep_idxs]

def prune(m1, m2):
    if isinstance(m1, C2f):      # C2f as a top conv
        m1 = m1.cv2

    if not isinstance(m2, list): # m2 is just one module
        m2 = [m2]

    for i, item in enumerate(m2):
        if isinstance(item, C2f) or isinstance(item, SPPF):
            m2[i] = item.cv1

    prune_conv(m1, m2)

for name, m in model.named_modules():
    if isinstance(m, Bottleneck):
        prune_conv(m.cv1, m.cv2)

seq = model.model
for i in range(3, 9):
    if i in [6, 4, 9]: continue
    prune(seq[i], seq[i+1])

detect:Detect = seq[-1]
last_inputs   = [seq[15], seq[18], seq[21]]
colasts       = [seq[16], seq[19], None]
for last_input, colast, cv2, cv3 in zip(last_inputs, colasts, detect.cv2, detect.cv3):
    prune(last_input, [colast, cv2[0], cv3[0]])
    prune(cv2[0], cv2[1])
    prune(cv2[1], cv2[2])
    prune(cv3[0], cv3[1])
    prune(cv3[1], cv3[2])

for name, p in yolo.model.named_parameters():
    p.requires_grad = True


# Save the pruned model correctly
torch.save(model.model.state_dict(), '/content/drive/MyDrive/HackIA/ModuleEdge/models/pruned_model_yolo.pt')


model.0.bn 8.109375 1.173828125 11.1171875 0.39892578125
model.1.bn 6.9140625 2.400390625 5.59765625 0.1265869140625
model.2.cv1.bn 5.6953125 0.7177734375 3.185546875 0.021087646484375
model.2.cv2.bn 2.849609375 0.5634765625 2.830078125 0.11328125
model.2.m.0.cv1.bn 4.1171875 1.28515625 5.2734375 0.109619140625
model.2.m.0.cv2.bn 2.974609375 1.23828125 4.8046875 0.03759765625
model.3.bn 1.2607421875 0.4248046875 2.708984375 0.031402587890625
model.4.cv1.bn 1.5107421875 0.2274169921875 1.58203125 0.0013256072998046875
model.4.cv2.bn 1.2373046875 0.356689453125 2.572265625 0.00991058349609375
model.4.m.0.cv1.bn 1.0048828125 0.44970703125 1.6728515625 0.06317138671875
model.4.m.0.cv2.bn 1.0673828125 0.42431640625 1.53515625 0.0014619827270507812
model.4.m.1.cv1.bn 0.9501953125 0.427001953125 2.28515625 0.027862548828125
model.4.m.1.cv2.bn 1.5087890625 0.626953125 2.57421875 0.0301666259765625
model.5.bn 1.7451171875 0.402099609375 1.7587890625 0.0002868175506591797
model.6.cv1.bn 3.132812

In [14]:
import torch
import torch.nn as nn
from torch.nn.utils import prune
from ultralytics import YOLO

# Load the YOLO model
model = YOLO("/content/drive/MyDrive/HackIA/ModuleEdge/models/best.pt")

def sparsity(model):
    # Return global model sparsity
    a, b = 0, 0
    for p in model.parameters():
        a += p.numel()
        b += (p == 0).sum().item()
    return b / a

pruning_param = 0.3

# Apply pruning to the model
for name, m in model.model.named_modules():
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        prune.l1_unstructured(m, name='weight', amount=pruning_param)  # prune
        prune.remove(m, 'weight')  # make permanent

print(f'Model pruned to {sparsity(model.model):.3g} global sparsity')
ckpt = {

            'model': model.model,
            'train_args': {},  # save as dict
}

# Save the pruned model's state_dict and other required attributes
torch.save(ckpt,'/content/model_pruned_3.pt')
model.val()


Model pruned to 0.299 global sparsity
Ultralytics YOLOv8.2.18 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


FileNotFoundError: '/content/fire_config.yaml' does not exist